# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [7]:
# your code here
from __future__ import print_function
import tensorflow.keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import RMSprop
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy


In [5]:
data = pd.read_csv('tic-tac-toe.csv')
data

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,False
954,o,x,o,x,x,o,x,o,x,False
955,o,x,o,x,o,x,x,o,x,False
956,o,x,o,o,x,x,x,o,x,False


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


In [8]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Convert the categorical values to numeric in all columns
label_encoder = LabelEncoder()

# Apply the LabelEncoder to all the columns of the dataframe
data_transformed = data.apply(label_encoder.fit_transform)
data_transformed.head()

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,2,2,2,2,1,1,2,1,1,1
1,2,2,2,2,1,1,1,2,1,1
2,2,2,2,2,1,1,1,1,2,1
3,2,2,2,2,1,1,1,0,0,1
4,2,2,2,2,1,1,0,1,0,1


In [43]:
# Separate the inputs and output
X = data_transformed.drop('class',axis=1)
y = data_transformed['class']

In [10]:
# Normalize the input data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [11]:
# Split the training and test data

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)

In [12]:
# Create a Sequencial model

model = Sequential()

In [13]:
# Add layers to the model. Relu as activation function for the middle layers. Softmax for the output layer.

# LAYER 1
model.add(Dense(128, input_dim = 9, activation='relu'))

# LAYER 2
model.add(Dropout(0.2)) # recalibrate the weights. Improve the model to not overfit.
model.add(Dense(64, activation='relu'))

# LAYER 3
model.add(Dropout(0.2)) 
model.add(Dense(32, activation='relu'))

# LAYER 4
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax')) #final output will be binary, so 2.

c:\Users\Francesco Corda\Documents\Aline\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
# Model Summary

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,048 (136.91 KB)

 Trainable params: 11,682 (45.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 23,366 (91.28 KB)

In [16]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [17]:
# Fit the training data

history = model.fit(X_train, y_train, epochs=50, batch_size=10, validation_data=(X_test, y_test))

Epoch 1/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.6409 - loss: 0.6563 - val_accuracy: 0.6510 - val_loss: 0.6346
Epoch 2/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6440 - loss: 0.6335 - val_accuracy: 0.6615 - val_loss: 0.6130
Epoch 3/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6692 - loss: 0.6018 - val_accuracy: 0.7031 - val_loss: 0.5968
Epoch 4/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6901 - loss: 0.5909 - val_accuracy: 0.7396 - val_loss: 0.5804
Epoch 5/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7338 - loss: 0.5695 - val_accuracy: 0.7604 - val_loss: 0.5627
Epoch 6/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7163 - loss: 0.5697 - val_accuracy: 0.7396 - val_loss: 0.5438
Epoch 7/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7493 - loss: 0.5081 - val_accuracy: 0.7396 - val_loss: 0.5206
Epoch 8/50
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8094 - loss: 0.4560 - val_accuracy: 0.7656 - val_loss:

In [21]:
# Evaluate the model with the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")
print(f"Test Loss: {loss:.2f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9212 - loss: 0.2458 
Test Accuracy: 0.92
Test Loss: 0.28


In [22]:
# Save the trained model
model.save('tic-tac-toe.keras')


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [27]:
# Load the saved model
from tensorflow.keras.models import load_model

model = load_model('tic-tac-toe.keras')

In [46]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   TL      958 non-null    int32
 1   TM      958 non-null    int32
 2   TR      958 non-null    int32
 3   ML      958 non-null    int32
 4   MM      958 non-null    int32
 5   MR      958 non-null    int32
 6   BL      958 non-null    int32
 7   BM      958 non-null    int32
 8   BR      958 non-null    int32
dtypes: int32(9)
memory usage: 33.8 KB


In [50]:
column_names = ['TL', 'TM', 'TR','ML','MM','MR','BL','BM','BR']  # Replace with your actual column names
X_test_df = pd.DataFrame(X_test, columns=column_names)

In [51]:
X_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   TL      192 non-null    float64
 1   TM      192 non-null    float64
 2   TR      192 non-null    float64
 3   ML      192 non-null    float64
 4   MM      192 non-null    float64
 5   MR      192 non-null    float64
 6   BL      192 non-null    float64
 7   BM      192 non-null    float64
 8   BR      192 non-null    float64
dtypes: float64(9)
memory usage: 13.6 KB


In [52]:
y_test_df = pd.Series(y_test, name='class')  #  

In [55]:
X_test_df

,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,0.5,0.5,0.5,0.0,1.0,1.0,1.0,0.5,1.0
1,0.5,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.5
2,0.5,1.0,0.0,1.0,1.0,0.0,0.5,1.0,0.5
3,0.0,1.0,0.5,1.0,1.0,0.5,1.0,0.5,0.5
4,0.5,0.0,1.0,1.0,0.5,0.0,0.0,1.0,0.5
...,...,...,...,...,...,...,...,...,...
187,0.5,0.0,0.5,1.0,1.0,1.0,0.0,1.0,0.5
188,1.0,0.5,1.0,1.0,0.5,1.0,0.5,0.5,0.0
189,1.0,1.0,1.0,0.5,0.5,0.0,0.0,0.0,0.0
190,1.0,1.0,1.0,0.5,0.5,0.0,0.0,1.0,0.5


In [61]:
X_test_df.iloc[list(random_indices)]

,TL,TM,TR,ML,MM,MR,BL,BM,BR
169,1.0,0.5,0.0,1.0,0.5,0.0,1.0,1.0,0.5
101,1.0,0.5,0.0,1.0,0.5,0.5,1.0,0.0,1.0
102,0.0,1.0,0.5,1.0,0.5,1.0,0.5,0.0,0.0
5,1.0,0.0,0.5,0.5,1.0,0.5,1.0,0.0,1.0
113,1.0,0.0,0.5,1.0,1.0,0.5,0.0,0.0,0.5


In [63]:

# Select a few random indices from the test dataset
num_samples = 5
random_indices = np.random.choice(len(X_test_df), size=num_samples, replace=False)

# Use the selected indices to get the corresponding rows from X_test and y_test
X_random_test = X_test_df.iloc[list(random_indices)]
y_random_true = y_test_df.iloc[list(random_indices)]

# Make predictions on the selected rows
y_random_pred = model.predict(X_random_test)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


In [64]:
# Convert predicted probabilities to class labels
y_random_pred_classes = np.argmax(y_random_pred, axis=1)

# Check if the predictions match the actual labels
for i in range(num_samples):
    print(f"Sample {i+1}:")
    print(f"  - True label: {y_random_true.iloc[i]}")
    print(f"  - Predicted class: {y_random_pred_classes[i]}")

    # Check if the prediction is correct
    if y_random_true.iloc[i] == y_random_pred_classes[i]:
        print("  - Prediction is correct")
    else:
        print("  - Prediction is incorrect")
    print()


Sample 1:
  - True label: 0
  - Predicted class: 0
  - Prediction is correct

Sample 2:
  - True label: 1
  - Predicted class: 1
  - Prediction is correct

Sample 3:
  - True label: 1
  - Predicted class: 1
  - Prediction is correct

Sample 4:
  - True label: 1
  - Predicted class: 1
  - Prediction is correct

Sample 5:
  - True label: 1
  - Predicted class: 1
  - Prediction is correct



## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [65]:
# Add layers to the model. Relu as activation function for the middle layers. Softmax for the output layer.

# LAYER 1
model.add(Dense(128, input_dim = 9, activation='relu'))

# LAYER 2
model.add(Dropout(0.2)) # recalibrate the weights. Improve the model to not overfit.
model.add(Dense(64, activation='relu'))

# LAYER 3
model.add(Dropout(0.2)) 
model.add(Dense(32, activation='relu'))

# LAYER 4
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax')) #final output will be binary, so 2.

c:\Users\Francesco Corda\Documents\Aline\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [75]:
# Compile the model with a custom learning rate
custom_optimizer = Adam(learning_rate=0.002)  # Adjust the learning rate as needed
model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [76]:
# Fit the training data

history = model.fit(X_train, y_train, epochs=60, batch_size=10, validation_data=(X_test, y_test))

Epoch 1/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9678 - loss: 0.1206 - val_accuracy: 0.9479 - val_loss: 0.2527
Epoch 2/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9701 - loss: 0.0899 - val_accuracy: 0.9115 - val_loss: 0.2312
Epoch 3/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9748 - loss: 0.1032 - val_accuracy: 0.9479 - val_loss: 0.2001
Epoch 4/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9747 - loss: 0.0526 - val_accuracy: 0.9375 - val_loss: 0.2128
Epoch 5/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9729 - loss: 0.1105 - val_accuracy: 0.9167 - val_loss: 0.1751
Epoch 6/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9652 - loss: 0.1007 - val_accuracy: 0.9635 - val_loss: 0.1847
Epoch 7/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9901 - loss: 0.0413 - val_accuracy: 0.9062 - val_loss: 0.3021
Epoch 8/60
77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9640 - loss: 0.0878 - val_accuracy: 0.9219 - val_loss:

In [77]:
# Evaluate the model with the test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")
print(f"Test Loss: {loss:.2f}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9630 - loss: 0.1448 
Test Accuracy: 0.95
Test Loss: 0.19


**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
## by increasing my epoch and learning rate.